In [1]:
import gzip
import itertools
import subprocess
import os
import re
import progressbar
import time
from itertools import izip_longest, izip, islice, tee, repeat

In [2]:
__author__='Maggie Ruimin Sun'

In [25]:
source = '/home/yaneng/RSun/Data/qiagen-colon/'
outdir = source + 'undetermined/'
readFile1 = 'QIAGEN-2959YJ_S2_L001_R1_001.fastq.gz'
readFile2 = 'QIAGEN-2959YJ_S2_L001_R2_001.fastq.gz'
gspFile = 'dhs-002z_primer5_refGene.txt'
read1_out = 'QIAGEN-2959YJ_S2_L001_R1_001_undetermined.fq'
read2_out = 'QIAGEN-2959YJ_S2_L001_R2_001_undetermined.fq'
index1_out = 'QIAGEN-2959YJ_S2_L001_I1_001_undetermined.fq'
index2_out = 'QIAGEN-2959YJ_S2_L001_I2_001_undetermined.fq'

In [4]:
fq1 = gzip.open(source+readFile1, 'rb')
reads1 = fq1.readlines()
fq1.close()
len(reads1)

6433396

In [5]:
primers = {}
primer_len = []
gsp1 = open(source+gspFile)
for row in gsp1:
    v = row.strip().split('\t')
    if len(v) < 2:
        break
    prim_seq = v[2]
    primer_len.append(len(prim_seq))
    v.remove(prim_seq)
    primers[prim_seq] = '\t'.join(v)
gsp1.close()
print min(primer_len), max(primer_len), sum(primer_len) / float(len(primer_len))

20 44 33.0655513827


#### For the first read mates, separate primers from the read sequences
For each read sequence, compare the first k-mer with the primer list, where k $\in$ (primer_len_min, primer_len_max). Once the primer is determined, separate the read sequence into two parts: one is the primer and the other is the DNA sequence of interest.

In [6]:
Nreads = len(reads1) / 4
primer_len_min = min(primer_len)
primer_len_max = max(primer_len)
primers_sorted = sorted(primers, key=len, reverse=True)

Define a function to check whether a primer has probability to match a subsequence by allowing at most 2 mismatches. 
* Given a primer, divide it into 3 non-overlapping parts, averagely.
* The function aims at checking if one of the three parts can be exactly mapped to a read sequence.
* If none of the three parts of a primer has exact match to the read, return 0 meaning that the primer has at least three mismatches to the read sequence. 

In [7]:
def hamming_distance_checker2(s, p):
    ls = len(p) / 3
    subsetp = [p[0:ls], p[ls:2*ls], p[2*ls:]]
    flag = 0
    for i in range(3):
        if s.find(subsetp[i]) >= 0:
            flag += 1
    return flag

#### For the second read mates, separate UMI+common sequence from the read sequences

In [11]:
fq2 = gzip.open(source+readFile2, 'rb')
reads2 = fq2.readlines()
len(reads2)

6433396

In [12]:
commonSeq = 'ATTGGAGTCCT'
len_cs = len(commonSeq)
len_MT = 12
st_read = len_MT+len_cs

For each sequence in the 1st read mates,
* Firstly, check the primers with exact matching.
* Secondly, if the read does not contain exactly-matched primers, consider the primer-read mapping with at most 2 mismatches allowed.


For each sequence in the 2nd read mates, the first 12-bp is the unique molecular index sequence, the subsequent 11-bp is the common sequence, and the rest is the read sequence of interest.

In [23]:
out_read1 = open(outdir+read1_out, 'w')
out_index1 = open(outdir+index1_out, 'w')
out_read2 = open(outdir+read2_out, 'w')
out_index2 = open(outdir+index2_out, 'w')
nf = 0
nr1, nr2, ni1, ni2 = [0,0,0,0]
max_Hamming_dist = 2
start_time = time.time()
bar = progressbar.ProgressBar()
for i in bar(range(Nreads)):
    header_i = reads1[4*i].strip()
    seq_i = reads1[4*i+1].strip()
    score_i = reads1[4*i+3]
    lr = len(seq_i)
    yes = 0
    diff_min = 2
    lpr = 0
    st = 0
    for pr_i in primers:
        loc_pr = seq_i[:50].find(pr_i)
        if loc_pr >= 0:
            st = loc_pr
            lpr = len(pr_i)
            pr_hi = primers[pr_i]
            pr_si = score_i[st:(st+lpr)]
            yes = 1
            break
            
    if yes == 0:        
        for pr in primers_sorted:
            flag = hamming_distance_checker2(seq_i, pr)            
            if flag == 0:
                continue
            k = len(pr)
            for j in range(51-k):
                kmer = seq_i[j:(j+k)]
                dist = k - sum([kmer[b]==pr[b] for b in range(len(kmer))])
                if dist <= max_Hamming_dist:
                    pr_i = kmer
                    pr_hi = primers[pr]
                    pr_si = score_i[j:(j+k)]
                    lpr = k
                    st = j
                    yes = 1
                    break
            if yes == 1:
                break

    if yes == 0:
        nf += 1
        out_index1.write(header_i+'\tchrN\nN\n+\n#\n')
        ni1 += 1
    else:
        seq_i = seq_i[(st+lpr):]
        score_i = score_i[(st+lpr):]
        out_index1.write(header_i+'\t'+pr_hi+'\n'+pr_i+'\n+\n'+pr_si+'\n')
        ni1 += 1
        
    if len(seq_i) == 0:
        seq_i = 'N' * 100
        score_i = '#' * 100 + '\n'
    out_read1.write(header_i+'\n'+seq_i+'\n+\n'+score_i)
    nr1+=1
    
    header2_i = reads2[4*i]
    seq2_i = reads2[4*i+1].strip()
    score2_i = reads2[4*i+3]
    lr2 = len(seq2_i)
    umi_seq = seq2_i[0:len_MT]
    umi_scr = score2_i[0:len_MT]
    seq2_i = seq2_i[st_read:]
    score2_i = score2_i[st_read:]
    out_read2.write(header2_i+seq2_i+'\n+\n'+score2_i)
    nr2 += 1
    out_index2.write(header2_i+umi_seq+'\n+\n'+umi_scr+'\n')
    ni2 += 1

out_read1.close()
out_index1.close()
out_read2.close()
out_index2.close()
end_time = time.time()
print 'Time elapsed:', (end_time-start_time) / 60.0, 'min'

Time elapsed: 52.5131923159 min


100% |########################################################################|


In [24]:
print nf, Nreads
print nr1, ni1, nr2, ni2

69176 1608349
1608349 1608349 1608349 1608349
